In [1]:
import tensorflow
import numpy as np
import pandas as pd
import os
import tensorflow.keras as keras
zip_file=keras.utils.get_file(
    fname="gamma_angle_position_5deg_xy.h5",
    origin="https://zenodo.org/record/846388/files/gamma_angle_position_5deg_xy.h5?download=1",
    extract=True

)
dirnam=os.path.join(os.path.dirname(zip_file), "gamma_angle_position_5deg_xy.h5")
import h5py
import h5py
zip_file


2060006240/2060006240 [==============================] - 70s 0us/step


'/root/.keras/datasets/gamma_angle_position_5deg_xy.h5'

In [2]:
x=h5py.File(zip_file)
layer_0_data = (np.array(h5py.File(zip_file)["layer_0"][:]))
layer_1_data = (np.array(h5py.File(zip_file)["layer_1"][:]))
layer_2_data = (np.array(h5py.File(zip_file)["layer_2"][:]))
energy=(np.array(h5py.File(zip_file)["energy"]))
energy=energy.reshape(energy.shape[0])
print(layer_0_data.shape)

nrm_l0=np.max(layer_0_data)
nrm_l1=np.max(layer_1_data)
nrm_l2=np.max(layer_2_data)
nrm_e=np.max(energy)
rng = np.random.default_rng()
def noise_add(layer_name, alpha, noise):
    noise_ln = rng.normal(alpha, noise ,layer_name.shape[0]*layer_name.shape[1]*layer_name.shape[2])
    noise_ln = noise_ln.reshape(layer_name.shape[0],layer_name.shape[1],layer_name.shape[2])
    layer_name = layer_name + noise_ln
    layer_name[layer_name<0] = 0
noise_add(layer_0_data,0,167)
noise_add(layer_1_data,0,167)
noise_add(layer_2_data,0,167)

(500000, 3, 96)


In [6]:
Layer_0_data_preprocessed=layer_0_data/nrm_l0
Layer_1_data_preprocessed=layer_1_data/nrm_l1
Layer_2_data_preprocessed=layer_2_data/nrm_l2
energy = energy / nrm_e
flatten_layer_0=Layer_0_data_preprocessed.reshape(500000,3*96)
flatten_layer_1=Layer_1_data_preprocessed.reshape(500000,12*12)
flatten_layer_2=Layer_2_data_preprocessed.reshape(500000,6*12)

In [7]:
from sklearn.model_selection import train_test_split
layer_0_train, layer_0_test, energy_train, energy_test,  layer_1_train,layer_1_test, layer_2_train, layer_2_test,= train_test_split(flatten_layer_0,  energy, flatten_layer_1, flatten_layer_2,  test_size=0.4, random_state=42)
layer_0_train_2D, layer_0_test_2D, energy_train, energy_test,  layer_1_train_2D,layer_1_test_2D, layer_2_train_2D, layer_2_test2D,= train_test_split(Layer_0_data_preprocessed,  energy, Layer_1_data_preprocessed, Layer_2_data_preprocessed,  test_size=0.4, random_state=42)

In [98]:
lc1_test=np.array([np.concatenate((layer_0_test, layer_1_test, layer_2_test),axis=1)  ])
lc1_train=np.array([np.concatenate((layer_0_train, layer_1_train, layer_2_train),axis=1)  ])
lc1_train=lc1_train.reshape(lc1_train.shape[1],lc1_train.shape[2])
lc1_test=lc1_test.reshape(lc1_test.shape[1],lc1_test.shape[2])

In [15]:
import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [116]:
def dat():
    x_train=lc1_train
    y_train=energy_train
    x_test=lc1_test
    y_test=energy_test
    return x_train, y_train, x_test, y_test
x_train,y_train,x_test,y_test=data()

In [118]:
def dnn_opt(x_train, y_train, x_test, y_test):
    
    inputs = layers.Input(shape=(144+(96*3)+72,), name="input_features")
    x = inputs
    for units in hidden_units:        
        x  = layers.Dense(units, activation=tf.nn.relu)(x)
        x  = layers.Dropout({{uniform(0, 1)}})(x)
    logits = layers.Dense(1, name="output")(x)
    model=keras.Model(inputs=inputs, outputs=logits, name="dnn")
    # Create the model.
    model.compile(
        optimizer=keras.optimizers.Adam({{uniform(0.001, 0.003)}}),
        loss=keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")#,
        #metrics=tf.keras.metrics.BinaryAccuracy(),
    )
    
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=50, restore_best_weights=True
    )
    
    history = model.fit(
        x_train,
        y_train,
        epochs=50,
        batch_size={{choice([512,1024,2028,4056])}},
        validation_split=0.2,
        callbacks=[early_stopping],
    )

    validation_acc=np.amin(history.history["val_acc"])
    return {'loss': validation_acc, 'status': STATUS_OK, 'model': model}, 

In [121]:
x_train,y_train,x_test,y_test=data()
best_run, best_model = optim.minimize(model=dnn_opt,data=dat, algo=tpe.suggest,max_evals=5,trials=Trials(),notebook_name='Hyperparam')

>>> Imports:
#coding=utf-8

try:
    import tensorflow
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import os
except:
    pass

try:
    import tensorflow.keras as keras
except:
    pass

try:
    import h5py
except:
    pass

try:
    import h5py
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import hyperas
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Adam': hp.uniform('Adam', 0.001, 0.003),
        'batch_size': hp.choice('batch_size', [512,1024,2028,4056]),
    }

>>> Data
 1: 
 2: x_train=lc1_train
 3: y_train=energy_train
 4: x_test=lc1_test
 5: y_test=energy_test
 

NameError: name 'lc1_train' is not defined